In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-157037",resource_group="aml-quickstarts-157037",subscription_id="2c48c51c-bd47-40d4-abbe-fb8eabd19c8c")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-157037
Azure region: southcentralus
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-157037


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

name = "StandardD2V2"
try:

    cluster = ComputeTarget(ws, name)
    print(f"{name} exists already")

except:

    config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cluster = ComputeTarget.create(ws, name, config)

cluster.wait_for_completion(show_output=True)

StandardD2V2 exists already

Running


In [5]:
from azureml.widgets import RunDetails
#from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core import Environment
from azureml.core import ScriptRunConfig

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": choice(0.001, 0.01, 0.1, 1, 10, 100),
    "max_iter": choice(25,50,75,100,125)
})
# Specify a Policy
policy = BanditPolicy(evaluation_interval=100, slack_factor=0.2, delay_evaluation=200)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#SKLearn_estimator = SKLearn(source_directory='./', compute_target=cluster, entry_script='train.py')
env = Environment.from_conda_specification(name='sklearn_env', file_path='./dependencies.yml')
train_config = ScriptRunConfig(source_directory='.', script='./train.py', compute_target=cluster, environment=env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
HyperDriveConfig = HyperDriveConfig(run_config=train_config,
                                    policy=policy,
                                    primary_metric_name="Accuracy",
                                    hyperparameter_sampling=ps,
                                    max_total_runs=500,
                                    max_concurrent_runs=4,
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
HyperDrive_run = exp.submit(HyperDriveConfig)
RunDetails(HyperDrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [15]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
run_top = HyperDrive_run.get_best_run_by_primary_metric()
run_top_metric = run_top.get_metrics()
params = run_top.get_details()['runDefinition']['arguments']

print("run_id:{}, C:{}, max_iter:{}, accuracy {}".format(run_top.id,params[1],params[3],run_top_metric['Accuracy']))

run_id:HD_38705366-7ab2-4c90-b387-5e338bdfe782_0, C:10, max_iter:75, accuracy 0.9142640364188164


In [23]:
model = run_top.register_model(model_name='hyperdrive-model',
                               model_path='./',
                               tags={'Method':'Hyperdrive'},
                               properties={'Accuracy':run_top_metric['Accuracy']})

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
### YOUR CODE HERE ###
df = TabularDatasetFactory.from_delimited_files(dataset)

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(df)

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=df,
    label_column_name='y',
    n_cross_validations=5)

In [11]:
# Submit your automl run

### YOUR CODE HERE ###
automl_experiment = Experiment(ws, "automl_experiment")
automl_run = automl_experiment.submit(config = automl_config, show_output = True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_5c2c4590-e111-4217-aa16-85d22db91a0b,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

Cannot serialize JSON possibly due to timestamp, retrying


In [12]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [14]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, best_automl_model = automl_run.get_output()

print(best_automl_run)

print(best_automl_run.get_metrics()['accuracy'])

best_automl_run.register_model(model_name = 'best_automl_experiment.pkl', model_path = './outputs/')

Run(Experiment: automl_experiment,
Id: AutoML_5c2c4590-e111-4217-aa16-85d22db91a0b_25,
Type: None,
Status: Completed)
0.9179059180576632


Model(workspace=Workspace.create(name='quick-starts-ws-157037', subscription_id='2c48c51c-bd47-40d4-abbe-fb8eabd19c8c', resource_group='aml-quickstarts-157037'), name=best_automl_experiment.pkl, id=best_automl_experiment.pkl:1, version=1, tags={}, properties={})

In [ ]:
# delete virtual machine
cluster.delete()